Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [67]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### Logistic

In [6]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Regularization
    regularizer = tf.nn.l2_loss(weights)
    loss_l2 = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss_l2)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.631283
Minibatch accuracy: 7.8%
Validation accuracy: 13.1%
Minibatch loss at step 500: 0.509164
Minibatch accuracy: 85.9%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.724155
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.491850
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.572323
Minibatch accuracy: 88.3%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.707439
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 3000: 0.707402
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Test accuracy: 88.6%


#### NN with L2

In [35]:
batch_size = 128
num_nodes = 1024
beta = 0.01

graph = tf.Graph()
with graph.as_default():


    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = {
        'input': tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes])),
        'output': tf.Variable(tf.truncated_normal([num_nodes,10]))
    }
    biases = {
        'input': tf.Variable(tf.zeros([num_nodes])),
        'output': tf.Variable(tf.zeros([num_labels]))
    }
    
    # Layers
    def end_to_end(x):
        l1 = tf.matmul(x, weights['input']) + biases['input']
        l2 = tf.nn.relu(l1)
        logits = tf.matmul(l2, weights['output']) + biases['output']
        return logits
    
    logits = end_to_end(tf_train_dataset)
   
    # Training computation.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    regularizer = beta * tf.nn.l2_loss(weights['input'])
    l2_loss = tf.reduce_mean(loss + regularizer)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(end_to_end(valid_dataset))
    test_prediction = tf.nn.softmax(end_to_end(test_dataset))

In [36]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 300.346252
Minibatch accuracy: 12.5%
Validation accuracy: 22.0%
Minibatch loss at step 500: 1.417120
Minibatch accuracy: 85.9%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 0.659773
Minibatch accuracy: 79.7%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 0.369539
Minibatch accuracy: 88.3%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 0.423880
Minibatch accuracy: 89.1%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 0.476723
Minibatch accuracy: 85.2%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 0.538901
Minibatch accuracy: 84.4%
Validation accuracy: 84.7%
Test accuracy: 91.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [38]:
num_steps = 3001
new_train_dataset = train_dataset[:200,:]
new_train_labels = train_labels[:200,:]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (new_train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = new_train_dataset[offset:(offset + batch_size), :]
        batch_labels = new_train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 3000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 347.096008
Minibatch accuracy: 7.8%
Validation accuracy: 31.2%
Minibatch loss at step 3000: 0.001389
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Test accuracy: 80.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

#### NN with L2 & Dropout

In [50]:
batch_size = 128
num_nodes = 1024

graph = tf.Graph()
with graph.as_default():


    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = {
        'input': tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes])),
        'output': tf.Variable(tf.truncated_normal([num_nodes,10]))
    }
    biases = {
        'input': tf.Variable(tf.zeros([num_nodes])),
        'output': tf.Variable(tf.zeros([num_labels]))
    }
    
    # Layers
    def end_to_end(x,dropout = 1.):
        l1 = tf.matmul(x, weights['input']) + biases['input']
        l2 = tf.nn.dropout(tf.nn.relu(l1),dropout)
        logits = tf.matmul(l2, weights['output']) + biases['output']
        return logits
    
    keep_prob = tf.placeholder(tf.float32)
    logits = end_to_end(tf_train_dataset, keep_prob)
   
    # Training computation.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    regularizer = beta * tf.nn.l2_loss(weights['input'])
    l2_loss = tf.reduce_mean(loss + regularizer)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(l2_loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(end_to_end(valid_dataset))
    test_prediction = tf.nn.softmax(end_to_end(test_dataset))

In [51]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 482.742523
Minibatch accuracy: 10.9%
Validation accuracy: 26.9%
Minibatch loss at step 500: 2.369040
Minibatch accuracy: 82.0%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 0.666174
Minibatch accuracy: 81.2%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 0.571176
Minibatch accuracy: 81.2%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 0.576781
Minibatch accuracy: 85.9%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 0.667366
Minibatch accuracy: 75.0%
Validation accuracy: 82.6%
Minibatch loss at step 3000: 0.708176
Minibatch accuracy: 78.9%
Validation accuracy: 83.0%
Test accuracy: 89.9%


##### Extreme overfitting case

In [52]:
num_steps = 3001
new_train_dataset = train_dataset[:200,:]
new_train_labels = train_labels[:200,:]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 518.434082
Minibatch accuracy: 9.4%
Validation accuracy: 26.1%
Minibatch loss at step 500: 2.945080
Minibatch accuracy: 76.6%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 0.739105
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 0.484082
Minibatch accuracy: 84.4%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 0.585849
Minibatch accuracy: 86.7%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 0.673846
Minibatch accuracy: 80.5%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 0.743323
Minibatch accuracy: 78.9%
Validation accuracy: 82.3%
Test accuracy: 89.5%


With dropout, the overfitting at minibatch accuracy significantly dropped (comparing to 100% without dropout). 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


#### Two Relu Layers
  - initial step: simply adding one more (linear -> relu) layer. After doing this, it's observed that the minibatch loss became 'nan' after some steps. After further investigation, I found that the loss instead of decreases at each step, it grew exponentially, and hence became 'nan' very quickly. 
  - these steps can be taken: learning rate adjustment, optimize weight initialization, better optimizer (AdaGrad, momentum, Hessian, etc)
    - use [Xavier initialization](http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization) by simply defining the std dev of initialized weights. This works! The NN has behaved normally now.
      - With beta=0.001: 
        Minibatch loss at step 3000: 0.345069
        Minibatch accuracy: 89.1%
        Validation accuracy: 89.0%
        Test accuracy: 94.6%
      - With beta=0.01:
        Minibatch loss at step 3000: 0.448697
        Minibatch accuracy: 86.7%
        Validation accuracy: 85.7%
        Test accuracy: 92.0%
    - Use learning rate decay:
        Minibatch loss at step 3000: 0.451181
        Minibatch accuracy: 85.9%
        Validation accuracy: 85.6%
        Test accuracy: 92.0%
    

In [73]:
batch_size = 128
beta = 0.01
num_nodes = {
    1: 2048,
    2: 1024
}

num_steps = 3001

graph = tf.Graph()
with graph.as_default():


    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # Applied Xavier initialization
    weights = {
        'input': tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes[1]],stddev = 1./math.sqrt(image_size * image_size))),
        'hidden': tf.Variable(tf.truncated_normal([num_nodes[1],num_nodes[2]],stddev = 1./math.sqrt(num_nodes[1]))),
        'output': tf.Variable(tf.truncated_normal([num_nodes[2],10],stddev = 1./math.sqrt(num_nodes[2])))
    }
    biases = {
        'input': tf.Variable(tf.zeros([num_nodes[1]])),
        'hidden': tf.Variable(tf.zeros([num_nodes[2]])),
        'output': tf.Variable(tf.zeros([num_labels]))
    }
    
    # Layers
    # 1.Linear -> 2.Relu -> 3.Linear -> 4.Relu -> 5.Linear -> Softmax
    def end_to_end(x):
        l1 = tf.matmul(x, weights['input']) + biases['input']
        l2 = tf.nn.relu(l1)
        l3 = tf.matmul(l2, weights['hidden']) + biases['hidden']
        l4 = tf.nn.relu(l3)
        logits = tf.matmul(l4, weights['output']) + biases['output']
        return logits
    
    logits = end_to_end(tf_train_dataset)
   
    # Training computation.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    regularizer = beta * (tf.nn.l2_loss(weights['input'])) #+ tf.nn.l2_loss(weights['hidden']) + tf.nn.l2_loss(weights['output'])) 
    l2_loss = tf.reduce_mean(loss + regularizer)

    # Optimizer.
    global_step = tf.Variable(0,trainable=False)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(
        learning_rate=0.5, 
        global_step=global_step, 
        decay_steps=num_steps, 
        decay_rate=0.96, 
        staircase = True
    )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(l2_loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(end_to_end(valid_dataset))
    test_prediction = tf.nn.softmax(end_to_end(test_dataset))

In [ ]:
train_accuracies = np.empty(num_steps, dtype=np.float32)
valid_accuracies = np.empty(num_steps, dtype=np.float32)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        #append data to accuracy arrays for plotting
        train_accuracies[step] = accuracy(predictions, batch_labels)
        valid_accuracies[step] = accuracy(valid_prediction.eval(), valid_labels)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % train_accuracies[step])
            print("Validation accuracy: %.1f%%" % valid_accuracies[step])
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.308004
Minibatch accuracy: 6.2%
Validation accuracy: 35.0%


In [ ]:
from matplotlib.pyplot import plt
%matplotlib inline